In [14]:
import torch
from torch.autograd import Variable
from tqdm import tqdm
import random
from sklearn.metrics import mean_squared_error, f1_score
import wandb

In [15]:
class MatrixFactorization(torch.nn.Module):
    ''' A simple neural network that predicts a rating for a user and item'''
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        
        # Create user embeddings: These are the latent factors for users 
        # that capture their preferences in which types of items they prefer.
        self.user_factors = torch.nn.Embedding(n_users, n_factors)

        # Create item embeddings: These are the latent factors for items 
        # that reflect what they are at an implicit level
        self.item_factors = torch.nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        # Multiply the user and item embeddings to predict the score
        return (self.user_factors(user)*self.item_factors(item)).sum(1)

In [16]:
# Get the data from Canvas or from https://grouplens.org/datasets/movielens/100k/ 
#
# From the README:
#
# MovieLens data sets were collected by the GroupLens Research Project
# at the University of Minnesota.
# 
# This data set consists of:
#        * 100,000 ratings (1-5) from 943 users on 1682 movies. 
#        * Each user has rated at least 20 movies. 
#        * Simple demographic info for the users (age, gender, occupation, zip)

training_user_item_rating_tuples = []
users = set()
items = set()
with open('ml-100k/u.train', 'rt') as f:
    for line in f:
        cols = line.split()
        user = int(cols[0])
        item = int(cols[1])
        rating = int(cols[2])
        training_user_item_rating_tuples.append((user, item, rating))
        users.add(user)
        items.add(item)
        
test_user_item_rating_tuples = []
with open('ml-100k/u.test', 'rt') as f:
    for line in f:
        cols = line.split()
        user = int(cols[0])
        item = int(cols[1])
        rating = int(cols[2])
        test_user_item_rating_tuples.append((user, item, rating))      

In [17]:
len(training_user_item_rating_tuples)

244

In [18]:
[x for x  in training_user_item_rating_tuples if x[0]==7]

[]

In [19]:
# We'll create a factorization model with 20-dimensinal latent factors for users and items
model = MatrixFactorization(306, 90, n_factors=10)

# Use Mean Squared Error (MSE) loss to decide how wrong our predictions are
loss_fn = torch.nn.MSELoss()
lr = 1e-2

# This is stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

num_epochs = 20

wandb.init(
    project="electrifind_fm",
    config={
        "num_epochs": num_epochs,
        "learning_rate": lr,
        "architecture": "Matrix Factorization",
    }
)

# Tell pytorch we're going to train the model
model.train()

for epoch in range(num_epochs):

    loss_sum = 0

    random.shuffle(training_user_item_rating_tuples)

    for step, data in enumerate(tqdm(training_user_item_rating_tuples)):
        optimizer.zero_grad()

        # Get user, item and rating data and put them in a pytorch Tensor object
        rating = Variable(torch.FloatTensor([data[2]]))
        user = Variable(torch.LongTensor([data[0]]))
        item = Variable(torch.LongTensor([data[1]]))

        # Predict the rating. Note that this is *implicitly* calling .forward(user, item)
        # The notation seems weird at first, but this was adopt to remind everyone
        # that nerural network are themselves _functions_ over their inputs!
        prediction = model(user, item)

        # The loss function (defined above) figures out how wrong the prediction was
        loss = loss_fn(prediction, rating)

        # Backpropagate the error in the loss through the network
        # to figure out what needs to change
        loss.backward()
        loss_sum += loss.item()

        # Update the weights in the network using our particular optimizer
        optimizer.step()

        if step % 61 == 0 and step > 0:
            wandb.log({"loss": loss_sum})
            loss_sum = 0

wandb.finish()



100%|██████████| 244/244 [00:00<00:00, 1395.58it/s]


100%|██████████| 244/244 [00:00<00:00, 1714.50it/s]


100%|██████████| 244/244 [00:00<00:00, 1776.47it/s]


100%|██████████| 244/244 [00:00<00:00, 1768.72it/s]


100%|██████████| 244/244 [00:00<00:00, 1760.88it/s]


100%|██████████| 244/244 [00:00<00:00, 2054.14it/s]

100%|██████████| 244/244 [00:00<00:00, 4093.41it/s]

100%|██████████| 244/244 [00:00<00:00, 3974.75it/s]


100%|██████████| 244/244 [00:00<00:00, 1702.06it/s]


100%|██████████| 244/244 [00:00<00:00, 1723.52it/s]


100%|██████████| 244/244 [00:00<00:00, 1628.88it/s]


100%|██████████| 244/244 [00:00<00:00, 1804.07it/s]


100%|██████████| 244/244 [00:00<00:00, 1742.59it/s]


100%|██████████| 244/244 [00:00<00:00, 1839.48it/s]


100%|██████████| 244/244 [00:00<00:00, 2142.70it/s]


100%|██████████| 244/244 [00:00<00:00, 1779.55it/s]


100%|██████████| 244/244 [00:00<00:00, 1759.79it/s]


100%|██████████| 244/244 [00:00<00:00, 1756.78it/s]


100%|██████████| 244/244 [00

loss,█▇▅▄▃▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,33.77287


In [20]:
model.user_factors(torch.LongTensor([233]))

tensor([[ 0.9157, -0.2126,  0.3072,  0.0800, -0.0550, -1.2706, -0.3423,  0.3854,
          0.6266, -0.1453]], grad_fn=<EmbeddingBackward0>)

In [21]:
model.item_factors(torch.LongTensor([1]))

tensor([[ 0.4613,  1.2089,  1.5593,  0.4276, -0.1740,  0.5365,  1.8357, -0.4351,
         -0.2109,  0.5791]], grad_fn=<EmbeddingBackward0>)

In [22]:
(model.user_factors(torch.LongTensor([233])) 
 * model.item_factors(torch.LongTensor([1]))).sum(1)

tensor([-1.0058], grad_fn=<SumBackward1>)

In [23]:
# Tell pytorch we're going to evaluate, so don't try to learn
model.eval()

pred_ratings = []
gold_ratings = []
for user, item, rating in tqdm(test_user_item_rating_tuples):

    # Get user and item and put them in a pytorch Tensor object
    user = Variable(torch.LongTensor([user]))
    item = Variable(torch.LongTensor([item]))

    # Predict the score again
    prediction = model(user, item)
    
    # Get the value as a python float object
    prediction = prediction.detach().numpy()[0]
    
    pred_ratings.append(prediction)
    gold_ratings.append(rating)


100%|██████████| 61/61 [00:00<00:00, 11767.66it/s]


In [24]:
print(mean_squared_error(gold_ratings, pred_ratings, squared=False))

5.076237692674947


/home/zim/.virtualenvs/SI630/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [32]:
pred_ratings = [round(x) for x in pred_ratings]

In [33]:
print(f1_score(gold_ratings, pred_ratings, average='micro'))

0.13114754098360656
